### Combining l3 and l4

##### Columns labels for l2,l3,l4 files

In [27]:
l2_columns = ["SAMPLE_NUMBER",
              "UTC_TIME",
              "DAY_OF_YEAR_WITH_FRACTIONS",
              "EPHEMERIS_SECONDS",
              "DISTANCE",
              "TRANSMIT_FREQUENCY_RAMP_REFERENCE_TIME",
              "TRANSMIT_FREQUENCY_CONSTANT_TERM",
              "TRANSMIT_FREQUENCY_LINEAR_TERM",
              "OBSERVED_X_BAND_ANTENNA_FREQUENCY",
              "PREDICTED_X_BAND_ANTENNA_FREQUENCY",
              "CORRECTION_OF_EARTH_ATMOSPHERE_PROPAGATION",
              "RESIDUAL_CALIBRATED_X_BAND_FREQUENCY_SHIFT",
              "SIGNAL_LEVEL_X_BAND",
              "DIFFERENTIAL_DOPPLER",
              "SIGMA_OBSERVED_X_BAND_ANTENNA_FREQUENCY",
              "SIGNAL_QUALITY_X_BAND",
              "SIGMA_SIGNAL_LEVEL_X_BAND"]

l3_columns = ["SAMPLE_NUMBER",
              "UTC_TIME",
              "EPHEMERIS_SECONDS",
              "RESIDUAL_CALIBRATED_X_BAND_FREQUENCY_SHIFT",
              "RESIDUAL_CALIBRATED_X_BAND_FREQUENCY_SHIFT_AFTER_BASELINE_FIT",
              "RECONSTRUCTED_TRANSMIT_FREQUENCY",
              "RADIUS",
              "SIGMA_RADIUS",
              "BENDING_ANGLE",
              "SIGMA_BENDING_ANGLE",
              "REFRACTIVE_INDEX",
              "REFRACTIVITY",
              "SIGMA_REFRACTIVITY",
              "SIGNAL_LEVEL",
              "DIFFERENTIAL_DOPPLER",
              "IMPACT_PARAMETER",
              "SIGMA_IMPACT_PARAMETER",
              "LONGITUDE",
              "LATITUDE"]

l4_columns = ["SAMPLE_NUMBER",
              "UTC_TIME",
              "EPHEMERIS_SECONDS",
              "RADIUS",
              "LATITUDE",
              "LONGITUDE",
              "GEOPOTENTIAL",
              "GEOPOTENTIAL_HEIGHT",
              "PRESSURE_LOWER_TEMPERATURE_AT_BOUNDARY",
              "SIGMA_PRESSURE_LOWER_TEMPERATURE_AT_BOUNDARY",
              "PRESSURE_MEDIUM_TEMPERATURE_AT_BOUNDARY",
              "SIGMA_PRESSURE_MEDIUM_TEMPERATURE_AT_BOUNDARY",
              "PRESSURE_HIGHER_TEMPERATURE_AT_BOUNDARY",
              "SIGMA_PRESSURE_HIGHER_TEMPERATURE_AT_BOUNDARY",
              "TEMPERATURE_LOWER_TEMPERATURE_AT_BOUNDARY",
              "SIGMA_TEMPERATURE_LOWER_TEMPERATURE_AT_BOUNDARY",
              "TEMPERATURE_MEDIUM_TEMPERATURE_AT_BOUNDARY",
              "SIGMA_TEMPERATURE_MEDIUM_TEMPERATURE_AT_BOUNDARY",
              "TEMPERATURE_HIGHER_TEMPERATURE_AT_BOUNDARY",
              "SIGMA_TEMPERATURE_HIGHER_TEMPERATURE_AT_BOUNDARY",
              "NUMBER_DENSITY",
              "SIGMA_NUMBER_DENSITY",
              "SOLAR_ZENITH_ANGLE",
              "LOCAL_SOLAR_TIME"]

##### Making a zip with (l3,l4) file pairs pertaining to the same occultation event.

In [33]:
##### Making a zip with (l3,l4) file pairs pertaining to the same occultation event.
import glob

path = '/home/dev/Desktop/Venus/Data'
directories = ['vcors_2001', 'vcors_2002', 'vcors_2003', 'vcors_2004']
subdirectories = ['l3', 'l4']
subsubdirectories = ['ingress','egress']

l3_l4_zip=[]
for dir1 in directories:
    for dir2 in subsubdirectories:
        l3_files = glob.glob(path+'/'+dir1+'/l3/'+dir2+'/*.csv')
        l4_files = glob.glob(path+'/'+dir1+'/l4/'+dir2+'/*.csv')
        
        for l3 in l3_files:
            for l4 in l4_files:
                temp1 = l3.split('/')[-1][0:11]
                temp2 = l4.split('/')[-1][0:11]
                if temp1==temp2:
                    l3_l4_zip.append([l3,l4])

for i in range(len(l3_l4_zip)):
    l3_path = l3_l4_zip[i][0]
    l4_path = l3_l4_zip[i][1]
    print(l3_path, l4_path)

/home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170526_205100_udsc64_l3_i_v10.csv /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170526_205100_udsc64_l4_ai_v10.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170731_060200_idsn32_l3_i_v10.csv /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170731_060200_idsn32_l4_ae_v10.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170731_060200_idsn32_l3_i_v10.csv /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170731_060200_idsn32_l4_ai_v10.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170709_060500_idsn32_l3_i_v10.csv /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170709_060500_idsn32_l4_ai_v10.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20160723_022600_udsc64_l3_i_v10.csv /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20160723_022600_udsc64_l4_ai_v10.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20160426_001800_udsc64_l3_i_v10.csv /home

##### Combining l3 and l4 files

In [34]:
import pandas as pd
import os
import glob

path = '/home/dev/Desktop/Venus/Data'
directories = ['vcors_2001', 'vcors_2002', 'vcors_2003', 'vcors_2004']
subdirectories = ['ingress', 'egress']
combo_dir = os.path.join(path, 'combo')
l3_l4_dir = os.path.join(combo_dir, 'l3_l4')

# Create the output directory if it doesn't exist
os.makedirs(l3_l4_dir, exist_ok=True)

# Function to find the nearest row in table B for each row in table A
def find_nearest(row, table_l4):
    diff = abs(table_l4.iloc[:, 2] - row.iloc[2])  # Assuming Column3 is at index 2
    index_of_nearest = diff.idxmin()
    return table_l4.iloc[index_of_nearest]

# Combine l3 and l4 files
def combine_l3_l4(l3_file, l4_file, output_file):
    table_l3 = pd.read_csv(l3_file)
    table_l4 = pd.read_csv(l4_file)
    nearest_rows = table_l3.apply(find_nearest, axis=1, args=(table_l4,)).reset_index(drop=True)
    table_l4['BA'] = nearest_rows.iloc[:, 8]
    table_l4['REFRACTIVE_INDEX'] = nearest_rows.iloc[:, 10]
    table_l4['REFRACTIVITY'] = nearest_rows.iloc[:, 11]
    table_l4['IMPACT_PARAMETER'] = nearest_rows.iloc[:, 15]
    table_l4['LATITUDE'] = nearest_rows.iloc[:, 17]
    table_l4['LONGITUDE'] = nearest_rows.iloc[:, 18]
    table_l4.drop(table_l4.columns[[0, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, 21, 22, 23]], axis=1, inplace=True)
    output_file_path = os.path.join(l3_l4_dir, output_file)
    table_l4.to_csv(output_file_path, header=True, index=False)

# Step 1: Generate pairs of l3 and l4 files
l3_l4_pairs = []
for directory in directories:
    for subdirectory in subdirectories:
        l3_files = glob.glob(os.path.join(path, directory, 'l3', subdirectory, '*.csv'))
        l4_files = glob.glob(os.path.join(path, directory, 'l4', subdirectory, '*.csv'))
        for l3_file in l3_files:
            l3_filename = os.path.splitext(os.path.basename(l3_file))[0]
            l3_event_id = l3_filename.split('_')[1]
            for l4_file in l4_files:
                l4_filename = os.path.splitext(os.path.basename(l4_file))[0]
                l4_event_id = l4_filename.split('_')[1]
                if l3_event_id == l4_event_id:
                    output_filename = f"{l3_filename}_l3_l4.csv"
                    l3_l4_pairs.append((l3_file, l4_file, output_filename))

# Step 2: Combine l3 and l4 files
for l3_file, l4_file, output_filename in l3_l4_pairs:
    combine_l3_l4(l3_file, l4_file, output_filename)

print(f"Combined {len(l3_l4_pairs)} l3 and l4 files.")

Combined data from /home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170526_205100_udsc64_l3_i_v10.csv and /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170526_205100_udsc64_l4_ai_v10.csv into /home/dev/Desktop/Venus/Data/combo/l3_l4/rs_20170526_i_l3_l4_combined.csv
Combined data from /home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170731_060200_idsn32_l3_i_v10.csv and /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170731_060200_idsn32_l4_ae_v10.csv into /home/dev/Desktop/Venus/Data/combo/l3_l4/rs_20170731_i_l3_l4_combined.csv
Combined data from /home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170731_060200_idsn32_l3_i_v10.csv and /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170731_060200_idsn32_l4_ai_v10.csv into /home/dev/Desktop/Venus/Data/combo/l3_l4/rs_20170731_i_l3_l4_combined.csv
Combined data from /home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170709_060500_idsn32_l3_i_v10.csv and /home/dev/Desktop/Venus/Data/vcors_200

#### Checking if the merging worked out properly

##### Number of columns in each individual file

In [35]:
import os
import pandas as pd

output_folder = '/home/dev/Desktop/Venus/Data/combo/l3_l4'

# Get the list of files in the output folder
files = os.listdir(output_folder)

for file in files:
    file_path = os.path.join(output_folder, file)
    
    try:
        # Read the file using pandas to get the dimensions
        df = pd.read_csv(file_path)
        num_rows, num_cols = df.shape
    
        print(f"File: {file}")
        print(f"Dimensions: {num_rows} rows, {num_cols} columns")
        print("----------------------")
    except pd.errors.ParserError as e:
        error_message = str(e)
        line_number = int(error_message.split('line ')[1].split(',')[0])
        print(f"Error parsing file: {file}")
        print(f"Error message: {error_message}")
        print(f"Line number causing the error: {line_number}")
        print("----------------------")

File: rs_20200323_i_l3_l4_combined.csv
Dimensions: 289 rows, 23 columns
----------------------
File: rs_20180421_i_l3_l4_combined.csv
Dimensions: 362 rows, 23 columns
----------------------
File: rs_20190521_e_l3_l4_combined.csv
Dimensions: 370 rows, 23 columns
----------------------
File: rs_20220424_e_l3_l4_combined.csv
Dimensions: 338 rows, 23 columns
----------------------
File: rs_20200220_e_l3_l4_combined.csv
Dimensions: 361 rows, 23 columns
----------------------
File: rs_20170321_i_l3_l4_combined.csv
Dimensions: 331 rows, 23 columns
----------------------
File: rs_20160723_i_l3_l4_combined.csv
Dimensions: 316 rows, 23 columns
----------------------
File: rs_20190726_i_l3_l4_combined.csv
Dimensions: 199 rows, 23 columns
----------------------
File: rs_20220505_i_l3_l4_combined.csv
Dimensions: 314 rows, 23 columns
----------------------
File: rs_20210505_i_l3_l4_combined.csv
Dimensions: 363 rows, 23 columns
----------------------
File: rs_20160518_i_l3_l4_combined.csv
Dimensions:

##### Checking if all the files have the same number of columns

In [36]:
import os
import pandas as pd

output_folder = '/home/dev/Desktop/Venus/Data/combo/l3_l4'

# Get the list of files in the output folder
files = os.listdir(output_folder)

num_cols = None  # Variable to store the number of columns

for file in files:
    file_path = os.path.join(output_folder, file)
    
    # Read the file using pandas to get the number of columns
    df = pd.read_csv(file_path)
    curr_num_cols = df.shape[1]  # Get the number of columns
    
    if num_cols is None:
        num_cols = curr_num_cols
    elif num_cols != curr_num_cols:
        print("Not all files have the same number of columns.")
        break

if num_cols is not None:
    print(f"All files have the same number of columns: {num_cols}.")


All files have the same number of columns: 23.


##### Printing out the column lables of the files in the combined dataset

In [37]:
import os
import pandas as pd

combo_folder = '/home/dev/Desktop/Venus/Data/combo/l3_l4'

# Get a list of files in the combo folder
file_list = os.listdir(combo_folder)

# Iterate over the files and print the column names
for file_name in file_list:
    file_path = os.path.join(combo_folder, file_name)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
print(f"Columns in the combined l3_l4 file:")
print(df.columns)
print("--------------------")


Columns in the combined l3_l4 file:
Index(['UTC_TIME', 'EPHEMERIS_SECONDS',
       'TEMPERATURE_MEDIUM_TEMPERATURE_AT_BOUNDARY',
       'SIGMA_TEMPERATURE_MEDIUM_TEMPERATURE_AT_BOUNDARY',
       'PRESSURE_MEDIUM_TEMPERATURE_AT_BOUNDARY',
       'SIGMA_PRESSURE_MEDIUM_TEMPERATURE_AT_BOUNDARY', 'LATITUDE',
       'LONGITUDE', 'NUMBER_DENSITY', 'SIGMA_NUMBER_DENSITY', 'RADIUS',
       'SIGMA_RADIUS', 'BENDING_ANGLE',
       'RESIDUAL_CALIBRATED_X_BAND_FREQUENCY_SHIFT',
       'RESIDUAL_CALIBRATED_X_BAND_FREQUENCY_SHIFT_AFTER_BASELINE_FIT',
       'RECONSTRUCTED_TRANSMIT_FREQUENCY', 'SIGMA_BENDING_ANGLE',
       'REFRACTIVE_INDEX', 'REFRACTIVITY', 'SIGMA_REFRACTIVITY',
       'SIGNAL_LEVEL', 'DIFFERENTIAL_DOPPLER', 'IMPACT_PARAMETER'],
      dtype='object')
--------------------


### Combining l2 and l3_l4

##### Making a zip of l2 and l3_l4 files

In [38]:
import glob

l2_path = '/home/dev/Desktop/Venus/Data'
directories = ['vcors_2001', 'vcors_2002', 'vcors_2003', 'vcors_2004']
subdirectories = ['l2']
l3_l4_path = '/home/dev/Desktop/Venus/Data/combo/l3_l4'
l2_l3_l4_zip = []

for dir1 in directories:
    l2_files = glob.glob(l2_path+'/'+dir1+'/l2/'+'/*.csv')
    l3_l4_files = glob.glob(l3_l4_path+'/*.csv')
    
    for l2 in l2_files:
        for l3_l4 in l3_l4_files:
            temp1 = l2.split('/')[-1][0:11]
            temp2 = l3_l4.split('/')[-1][0:11]
            if temp1 == temp2:
                l2_l3_l4_zip.append([l2, l3_l4])

for i in range(len(l2_l3_l4_zip)):
    l2_file_path = l2_l3_l4_zip[i][0]
    l3_l4_file_path = l2_l3_l4_zip[i][1]
    print(l2_file_path, l3_l4_file_path)


/home/dev/Desktop/Venus/Data/vcors_2001/l2/rs_20170731_060200_idsn32_l2_v10.csv /home/dev/Desktop/Venus/Data/combo/l3_l4/rs_20170731_i_l3_l4_combined.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l2/rs_20170526_205100_udsc64_l2_v10.csv /home/dev/Desktop/Venus/Data/combo/l3_l4/rs_20170526_i_l3_l4_combined.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l2/rs_20170526_205100_udsc64_l2_v10.csv /home/dev/Desktop/Venus/Data/combo/l3_l4/rs_20170526_e_l3_l4_combined.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l2/rs_20160415_031000_udsc64_l2_v10.csv /home/dev/Desktop/Venus/Data/combo/l3_l4/rs_20160415_i_l3_l4_combined.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l2/rs_20160415_031000_udsc64_l2_v10.csv /home/dev/Desktop/Venus/Data/combo/l3_l4/rs_20160415_e_l3_l4_combined.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l2/rs_20170504_193200_udsc64_l2_v10.csv /home/dev/Desktop/Venus/Data/combo/l3_l4/rs_20170504_e_l3_l4_combined.csv
/home/dev/Desktop/Venus/Data/vcors_2001/l2/rs_20170504_193200_udsc64_l2_v10.

##### Combining l2 and l3_l4 files

In [40]:
import pandas as pd
import os
import glob

path = '/home/dev/Desktop/Venus/Data'
directories = ['vcors_2001', 'vcors_2002', 'vcors_2003', 'vcors_2004']
subdirectories = ['l2']
combo_dir = os.path.join(path, 'combo')
l2_l3_l4_dir = os.path.join(combo_dir, 'l2_l3_l4')

# Create the output directory if it doesn't exist
os.makedirs(l2_l3_l4_dir, exist_ok=True)

# Combine l2 and l3_l4 files
def combine_l2_l3_l4(l2_file, l3_l4_file, output_file):
    table_l2 = pd.read_csv(l2_file)
    table_l3_l4 = pd.read_csv(l3_l4_file)
    table_l3_l4['Signal_level'] = table_l2.iloc[:, 12]  # Assuming Signal_level is at index 12

    output_file_path = os.path.join(l2_l3_l4_dir, output_file)
    table_l3_l4.to_csv(output_file_path, header=True, index=False)

# Step 1: Generate pairs of l2 and l3_l4 files
l2_l3_l4_pairs = []
for directory in directories:
    for subdirectory in subdirectories:
        l2_files = glob.glob(os.path.join(path, directory, subdirectory, '*.csv'))
        l3_l4_files = glob.glob(os.path.join(l3_l4_dir, '*.csv'))
        for l2_file in l2_files:
            l2_filename = os.path.splitext(os.path.basename(l2_file))[0]
            l2_event_id = l2_filename.split('_')[1]
            for l3_l4_file in l3_l4_files:
                l3_l4_filename = os.path.splitext(os.path.basename(l3_l4_file))[0]
                l4_event_id = l3_l4_filename.split('_')[1]
                if l2_event_id == l4_event_id:
                    output_filename = f"{l2_filename}_l3_l4.csv"
                    l2_l3_l4_pairs.append((l2_file, l3_l4_file, output_filename))

# Step 2: Combine l2 and l3_l4 files
for l2_file, l3_l4_file, output_filename in l2_l3_l4_pairs:
    combine_l2_l3_l4(l2_file, l3_l4_file, output_filename)

print(f"Combined {len(l2_l3_l4_pairs)} l2 and l3_l4 files.")

Combined data from /home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170526_205100_udsc64_l3_i_v10.csv and /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170526_205100_udsc64_l4_ai_v10.csv into /home/dev/Desktop/Venus/Data/combo/l2_l3_l4/rs_20170526_il2_l3_l4_combined.csv
Combined data from /home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170731_060200_idsn32_l3_i_v10.csv and /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170731_060200_idsn32_l4_ae_v10.csv into /home/dev/Desktop/Venus/Data/combo/l2_l3_l4/rs_20170731_il2_l3_l4_combined.csv
Combined data from /home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170731_060200_idsn32_l3_i_v10.csv and /home/dev/Desktop/Venus/Data/vcors_2001/l4/ingress/rs_20170731_060200_idsn32_l4_ai_v10.csv into /home/dev/Desktop/Venus/Data/combo/l2_l3_l4/rs_20170731_il2_l3_l4_combined.csv
Combined data from /home/dev/Desktop/Venus/Data/vcors_2001/l3/ingress/rs_20170709_060500_idsn32_l3_i_v10.csv and /home/dev/Desktop/Venus

#### Checking if the merging worked out properly

##### Number of columns in each individual file

In [41]:
import os
import pandas as pd

output_folder = '/home/dev/Desktop/Venus/Data/combo/l2_l3_l4'

# Get the list of files in the output folder
files = os.listdir(output_folder)

for file in files:
    file_path = os.path.join(output_folder, file)
    
    try:
        # Read the file using pandas to get the dimensions
        df = pd.read_csv(file_path)
        num_rows, num_cols = df.shape
    
        print(f"File: {file}")
        print(f"Dimensions: {num_rows} rows, {num_cols} columns")
        print("----------------------")
    except pd.errors.ParserError as e:
        error_message = str(e)
        line_number = int(error_message.split('line ')[1].split(',')[0])
        print(f"Error parsing file: {file}")
        print(f"Error message: {error_message}")
        print(f"Line number causing the error: {line_number}")
        print("----------------------")

File: rs_20160426_el2_l3_l4_combined.csv
Dimensions: 323 rows, 32 columns
----------------------
File: rs_20200209_el2_l3_l4_combined.csv
Dimensions: 365 rows, 32 columns
----------------------
File: rs_20220115_el2_l3_l4_combined.csv
Dimensions: 364 rows, 32 columns
----------------------
File: rs_20200528_il2_l3_l4_combined.csv
Dimensions: 387 rows, 32 columns
----------------------
File: rs_20220608_il2_l3_l4_combined.csv
Dimensions: 0 rows, 32 columns
----------------------
File: rs_20170606_il2_l3_l4_combined.csv
Dimensions: 368 rows, 32 columns
----------------------
File: rs_20200220_il2_l3_l4_combined.csv
Dimensions: 363 rows, 32 columns
----------------------
File: rs_20220323_el2_l3_l4_combined.csv
Dimensions: 352 rows, 32 columns
----------------------
File: rs_20220505_il2_l3_l4_combined.csv
Dimensions: 314 rows, 32 columns
----------------------
File: rs_20180502_il2_l3_l4_combined.csv
Dimensions: 357 rows, 32 columns
----------------------
File: rs_20200722_el2_l3_l4_comb

##### Checking if all the files have the same number of columns

In [42]:
import os
import pandas as pd

output_folder = '/home/dev/Desktop/Venus/Data/combo/l2_l3_l4'

# Get the list of files in the output folder
files = os.listdir(output_folder)

num_cols = None  # Variable to store the number of columns

for file in files:
    file_path = os.path.join(output_folder, file)
    
    # Read the file using pandas to get the number of columns
    df = pd.read_csv(file_path)
    curr_num_cols = df.shape[1]  # Get the number of columns
    
    if num_cols is None:
        num_cols = curr_num_cols
    elif num_cols != curr_num_cols:
        print("Not all files have the same number of columns.")
        break

if num_cols is not None:
    print(f"All files have the same number of columns: {num_cols}.")


All files have the same number of columns: 32.


##### Printing out the column lables of the files in the combined dataset

In [43]:
import os
import pandas as pd

combo_folder = '/home/dev/Desktop/Venus/Data/combo/l2_l3_l4'

# Get a list of files in the combo folder
file_list = os.listdir(combo_folder)

# Iterate over the files and print the column names
for file_name in file_list:
    file_path = os.path.join(combo_folder, file_name)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
print(f"Columns in the combined l3_l4 file:")
print(df.columns)
print("--------------------")


Columns in the combined l3_l4 file:
Index(['SAMPLE_NUMBER', 'UTC_TIME', 'EPHEMERIS_SECONDS', 'RADIUS', 'LATITUDE',
       'LONGITUDE', 'GEOPOTENTIAL', 'GEOPOTENTIAL_HEIGHT',
       'PRESSURE_LOWER_TEMPERATURE_AT_BOUNDARY',
       'SIGMA_PRESSURE_LOWER_TEMPERATURE_AT_BOUNDARY',
       'PRESSURE_MEDIUM_TEMPERATURE_AT_BOUNDARY',
       'SIGMA_PRESSURE_MEDIUM_TEMPERATURE_AT_BOUNDARY',
       'PRESSURE_HIGHER_TEMPERATURE_AT_BOUNDARY',
       'SIGMA_PRESSURE_HIGHER_TEMPERATURE_AT_BOUNDARY',
       'TEMPERATURE_LOWER_TEMPERATURE_AT_BOUNDARY',
       'SIGMA_TEMPERATURE_LOWER_TEMPERATURE_AT_BOUNDARY',
       'TEMPERATURE_MEDIUM_TEMPERATURE_AT_BOUNDARY',
       'SIGMA_TEMPERATURE_MEDIUM_TEMPERATURE_AT_BOUNDARY',
       'TEMPERATURE_HIGHER_TEMPERATURE_AT_BOUNDARY',
       'SIGMA_TEMPERATURE_HIGHER_TEMPERATURE_AT_BOUNDARY', 'NUMBER_DENSITY',
       'SIGMA_NUMBER_DENSITY', 'SOLAR_ZENITH_ANGLE', 'LOCAL_SOLAR_TIME',
       'BENDING_ANGLE', 'SIGMA_BENDING_ANGLE', 'REFRACTIVE_INDEX',
       'REFRACTIVI